In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
transform = transforms.ToTensor()

train_data = datasets.MNIST(root = 'mnist_train', train= True, transform= transform, download=True)
test_data = datasets.MNIST(root = 'mnist_test', train= False, transform= transform, download=True)

print("Train Data Size: {}".format(len(train_data)))
print("Test Data Size: {}".format(len(test_data)))

Train Data Size: 60000
Test Data Size: 10000


In [3]:
from torch.nn.modules.activation import ReLU

class CnnModel(nn.Module):
  def __init__(self):
    super(CnnModel, self).__init__()
        # 첫번째층
        # ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2))

    # 두번째층
    # ImgIn shape=(?, 14, 14, 32)
    #    Conv      ->(?, 14, 14, 64)
    #    Pool      ->(?, 7, 7, 64)
    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2))

    # 전결합층 7x7x64 inputs -> 10 outputs
    self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)

    # 전결합층 한정으로 가중치 초기화
    torch.nn.init.xavier_uniform_(self.fc.weight)

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    # 전결합층을 위해서 Flatten
    out = out.view(out.size(0), -1)   
    out = self.fc(out)

    return out

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
model = CnnModel().to(device)

cpu


In [4]:
learning_rate = 0.001

criterion = torch.nn.CrossEntropyLoss().to(device)    # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [5]:
epoch = 15
batch_size = 100

data_loader = torch.utils.data.DataLoader(dataset=train_data,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

total_batch = len(data_loader)

print(total_batch)

600


In [6]:
for e in range(epoch):
    avg_cost = 0

    # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y는 레이블.
    for X, Y in data_loader: 
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(e + 1, avg_cost))

[Epoch:    1] cost = 0.231674239
[Epoch:    2] cost = 0.0591264851
[Epoch:    3] cost = 0.043221157
[Epoch:    4] cost = 0.0351576097
[Epoch:    5] cost = 0.0275410283
[Epoch:    6] cost = 0.0235254485
[Epoch:    7] cost = 0.0203048754
[Epoch:    8] cost = 0.0169209484
[Epoch:    9] cost = 0.0134118348
[Epoch:   10] cost = 0.0110896016
[Epoch:   11] cost = 0.00968120713
[Epoch:   12] cost = 0.00958529767
[Epoch:   13] cost = 0.00718325656
[Epoch:   14] cost = 0.00618957868
[Epoch:   15] cost = 0.0050613815


In [7]:
# 학습을 진행하지 않을 것이므로 torch.no_grad()
with torch.no_grad():
    X_test = test_data.test_data.view(len(test_data), 1, 28, 28).float().to(device)
    Y_test = test_data.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

/opt/homebrew/lib/python3.11/site-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/opt/homebrew/lib/python3.11/site-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.984499990940094
